In [1]:
# Scripts

from models import Base
from utils import read_upload, search_books, move_books, author_overview_ddl, customer_overview_ddl, move_inventory_ddl
from db import get_engine, server_connection


# Libraries

from sqlalchemy import text
from sqlalchemy.orm import Session
import pandas as pd


# Setting up a server connection and creating a database

server_name = "Alstation"
database_name = "bookstore"
username = "sa"
password = "D@tabases"

server_engine = get_engine(server_name, username, password)
server_connection(server_engine, database_name)
engine = get_engine(server_name, username, password, database_name)
print(f"\nConnection to {database_name} established successfully!")


# Create and fill tables

views = ["author_overview", "customer_overview"]
Base.metadata.create_all(
    engine,
    tables=[table for table in Base.metadata.tables.values() if table.name not in views]
)
read_upload(Base, engine)


# Create views

with engine.connect() as conn:
    conn.execute(text("drop view if exists author_overview"))
    conn.execute(text("drop view if exists customer_overview"))
    conn.commit()
    conn.execute(author_overview_ddl)
    conn.execute(customer_overview_ddl)
    conn.commit()
    print("Views created successfully!")

# Create stored procedure

with engine.connect() as conn:
    conn.execute(text("drop procedure if exists MoveInventory"))
    conn.execute(move_inventory_ddl)
    conn.commit()
    print("Stored procedure created successfully!")


Connection to bookstore established successfully!
Views created successfully!
Stored procedure created successfully!


In [ ]:
# Access the author overview

author_overview_df = pd.read_sql(
    "select * from author_overview",
    con=engine
)
display(author_overview_df)

In [ ]:
# Access the customer overview

customer_overview_df = pd.read_sql(
    "select * from customer_overview",
    con=engine
)
display(customer_overview_df.sort_values(by="total_amount", ascending=False))

In [ ]:
# Search for books

session = Session(engine)
result = search_books(session, input("Search book: "))
display(result)

In [9]:
# Move books between stores

before, info, after = move_books(engine)
print("BEFORE:")
display(before)
print(info)
print("\nAFTER:")
display(after)

BEFORE:


,store_id,isbn,amount
0,1,9780765376671,15
1,4,9780765376671,14



    Inventory relocation in progress ...
    Amount x ISBN: 5 x 9780765376671
    From Store ID: 1
    To Store ID: 4
    

AFTER:


,store_id,isbn,amount
0,1,9780765376671,10
1,4,9780765376671,19
